In [1]:
import argparse
import time
import csv
import pickle
import operator
import datetime
import os

In [11]:
dataset = './test_gowalla_srgnn.csv'
import pandas as pd
test=pd.read_csv(dataset,sep=',',header=0, names=['session_id', 'timestamp','item_id','ct'])

ParserError: Too many columns specified: expected 4 and found 1

In [16]:
test.to_csv('train_gowalla_srgnn.csv',index=False)

In [32]:
t=pd.read_csv('test_gowalla_srgnn.csv')

NameError: name 'pd' is not defined

In [18]:
t.head()

,session_id,timestamp,item_id,ct
0,1,2014-04-07T10:54:09.868Z,214536500,0
1,1,2014-04-07T10:54:46.998Z,214536506,0
2,1,2014-04-07T10:57:00.306Z,214577561,0
3,2,2014-04-07T13:56:37.614Z,214662742,0
4,2,2014-04-07T13:57:19.373Z,214662742,0


In [12]:
dataset = 'test_gowalla_srgnn.csv'
with open(dataset, "r") as f:
    reader = csv.DictReader(f, delimiter='\t')
    print(reader)
    sess_clicks = {}
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    for data in reader:
        sessid = data['session_id']
        # 新的session
        if curdate and not curid == sessid:
            date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%d %H:%M:%S'))
            sess_date[curid] = date # 记录最后一个时间
        curid = sessid # 新session
        item = data['item_id'], int(data['timeframe'])# item,time
        curdate = ''
        curdate = data['eventdate'] #取时间
        if sessid in sess_clicks:
            sess_clicks[sessid] += [item]
        else:
            sess_clicks[sessid] = [item] # 变成item列表
        ctr += 1
    date = time.mktime(time.strptime(curdate[:19], '%Y-%m-%d %H:%M:%S'))
    for i in list(sess_clicks):
        sorted_clicks = sorted(sess_clicks[i], key=operator.itemgetter(1))
        sess_clicks[i] = [c[0] for c in sorted_clicks] # 用时间排序
    sess_date[curid] = date # 最后一个
print("-- Reading data @ %ss" % datetime.datetime.now())


-- Reading data @ 2022-04-02 13:30:29.551521s


In [13]:
dates = list(sess_date.items())
tra_sess = sorted(dates, key=operator.itemgetter(1)) 

In [14]:
print(len(tra_sess))    # 186670    # 7966257
print(tra_sess)

10000
[('116', 1236922612.0), ('409', 1236941128.0), ('184', 1236946491.0), ('3374', 1237004110.0), ('244', 1237040806.0), ('3157', 1237086676.0), ('3473', 1237089972.0), ('3211', 1237107980.0), ('3052', 1237192345.0), ('3117', 1237258487.0), ('11353', 1238373442.0), ('2592', 1238477207.0), ('141', 1239261102.0), ('38047', 1240222198.0), ('12420', 1240702000.0), ('3366', 1241140882.0), ('17818', 1241240032.0), ('438', 1241572757.0), ('196', 1242264846.0), ('2309', 1242452062.0), ('45991', 1242715002.0), ('3215', 1242970787.0), ('5031', 1242984803.0), ('39867', 1243647491.0), ('1952', 1244522290.0), ('3195', 1245479480.0), ('12985', 1246498697.0), ('267', 1246629952.0), ('3917', 1247397266.0), ('5840', 1247652887.0), ('6198', 1247899355.0), ('4155', 1248083192.0), ('39311', 1248118030.0), ('2893', 1248163950.0), ('3362', 1248179852.0), ('2523', 1248511412.0), ('26681', 1248732899.0), ('1778', 1248880455.0), ('3349', 1249018156.0), ('334', 1249167754.0), ('1674', 1251678341.0), ('7529', 

In [15]:
item_dict = {}
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 5:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print(item_ctr)     # 43098, 37484
    return train_ids, train_dates, train_seqs

In [16]:
tra_ids, tra_dates, tra_seqs = obtian_tra()

53863


In [20]:
def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    flag=0
    for id, seq, date in zip(range(len(iseqs)), iseqs, idates):
        
        endd=min(len(seq)-4,2)
        beginn=max(0,len(seq)-endd-99)
        for i in range(1, endd):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[beginn:-i]]
            out_dates += [date]
            ids += [id]
            print(len(out_seqs[-1]))
    return out_seqs, out_dates, labs, ids

In [21]:
tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_seqs, tra_dates)

100
100
100
100
100
100
100
12
100
100
7
100
100
20
45
100
9
100
100
100
9
100
100
13
100
100
100
100
100
16
24
7
35
17
28
100
10
63
100
100
56
57
92
100
100
30
100
100
100
100
100
9
10
51
100
23
42
43
98
7
25
62
75
100
18
6
46
60
15
71
31
34
68
100
100
33
100
32
15
40
19
100
100
100
100
100
100
100
100
91
13
100
5
46
81
11
100
70
27
55
13
100
19
7
20
100
100
100
6
67
8
53
32
7
5
62
100
8
15
67
82
100
13
100
5
24
66
100
50
11
47
100
100
100
21
8
10
100
70
15
11
35
100
5
100
32
70
30
7
100
100
100
100
19
55
100
61
5
100
25
31
100
9
99
7
100
6
100
35
100
100
100
36
45
36
100
83
59
100
100
7
5
100
100
100
31
33
9
28
5
85
39
20
100
48
9
8
84
7
100
63
10
93
24
8
5
63
41
24
72
34
41
100
7
16
5
100
100
17
100
16
10
100
89
21
8
100
100
37
100
100
48
100
8
12
21
100
23
47
8
59
44
25
100
100
19
32
10
21
78
5
43
100
100
100
12
30
100
6
45
30
27
13
26
100
71
100
67
100
19
21
33
83
29
33
50
65
24
7
28
11
100
100
48
29
100
23
45
5
43
5
14
10
10
9
10
18
23
12
51
100
67
18
54
100
41
100
62
9
100
10
23

8
40
100
65
40
18
47
10
47
18
11
6
32
11
9
24
9
9
12
62
10
5
7
35
100
9
8
70
6
13
8
22
37
21
8
8
12
100
17
12
14
6
10
23
48
46
38
7
17
63
86
13
15
22
24
100
7
17
5
58
58
21
41
15
19
52
25
6
8
6
18
26
14
13
10
57
63
7
6
67
34
16
31
12
100
71
14
7
29
5
5
22
24
22
62
17
11
30
20
15
80
5
6
14
27
15
24
14
16
16
7
7
41
30
8
16
57
17
67
8
7
18
26
100
11
9
7
10
14
5
10
20
39
6
30
59
41
38
6
13
91
22
22
5
100
82
22
79
100
52
5
100
24
41
5
16
9
9
11
11
91
5
8
24
46
38
51
11
100
32
17
19
5
35
19
12
39
10
29
11
5
75
37
7
24
47
5
7
9
28
8
89
30
46
5
5
53
11
100
10
8
100
32
49
8
91
50
11
38
15
8
12
100
100
10
15
10
13
22
16
60
21
14
14
13
44
24
91
100
57
39
6
7
12
33
39
38
6
5
17
25
75
9
38
15
33
16
35
39
5
8
22
50
16
24
19
49
38
100
6
7
16
5
6
41
15
31
13
68
13
40
70
16
22
5
45
13
28
63
5
36
31
8
100
7
11
17
96
8
13
23
100
15
36
13
15
23
20
15
5
12
15
10
21
100
100
21
8
10
33
7
11
46
23
7
36
100
6
19
6
12
6
6
19
42
13
6
10
31
36
11
22
19
6
14
42
31
50
8
5
9
11
11
22
6
9
82
15
81
32
15
6
56
11
75
35

100
10
9
5
10
10
8
29
7
40
8
13
20
10
7
11
6
6
10
14
6
7
10
17
5
6
5
11
6
17
8
29
16
12
16
5
10
13
12
16
7
79
9
5
12
18
69
56
5
5
11
15
67
8
7
7
7
12
46
8
10
8
5
7
8
16
8
9
11
11
11
6
17
12
44
8
5
5
20
6
13
13
9
9
6
21
11
18
9
14
7
7
13
9
6
51
39
15
7
18
10
11
9
5
5
17
5
9
5


In [22]:
len(tr_seqs)

9126

In [23]:
tra = (tr_seqs, tr_labs)
if not os.path.exists('sample'):
    os.makedirs('sample')
pickle.dump(tra, open('sample/test20.txt', 'wb'))
pickle.dump(tra_seqs, open('sample/all_test_seq20.txt', 'wb'))
print('Done.')

Done.


In [50]:
print(tra)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

